In [62]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/milkquality/milknew.csv


In [63]:
!python -m pip install --upgrade pip

In [64]:
!pip install --upgrade scikit-learn

In [92]:
!pip show scikit-learn

Name: scikit-learn
Version: 1.5.1
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /opt/conda/lib/python3.10/site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: bayesian-optimization, Boruta, category-encoders, cesium, eli5, esda, fastai, gplearn, hep-ml, hmmlearn, hpsklearn, hypertools, imbalanced-learn, kmapper, kmodes, librosa, lime, mapclassify, mlxtend, nilearn, pyLDAvis, pynndescent, qudida, rgf-python, scattertext, scikit-learn-intelex, scikit-optimize, scikit-plot, segregation, shap, sklearn-pandas, spopt, spreg, TPOT, tsfresh, umap-learn, vecstack, woodwork, yellowbrick


In [66]:
df=pd.read_csv("/kaggle/input/milkquality/milknew.csv")

In [67]:
df.head()

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour,Grade
0,6.6,35,1,0,1,0,254,high
1,6.6,36,0,1,0,1,253,high
2,8.5,70,1,1,1,1,246,low
3,9.5,34,1,1,0,1,255,low
4,6.6,37,0,0,0,0,255,medium


In [68]:
x=df.iloc[:,:-1]
x

,pH,Temprature,Taste,Odor,Fat,Turbidity,Colour
0,6.6,35,1,0,1,0,254
1,6.6,36,0,1,0,1,253
2,8.5,70,1,1,1,1,246
3,9.5,34,1,1,0,1,255
4,6.6,37,0,0,0,0,255
...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247
1055,6.7,38,1,0,1,0,255
1056,3.0,40,1,1,1,1,255
1057,6.8,43,1,0,1,0,250


In [69]:
y=df['Grade']
y

0         high
1         high
2          low
3          low
4       medium
         ...  
1054    medium
1055      high
1056       low
1057      high
1058       low
Name: Grade, Length: 1059, dtype: object

In [70]:
y=y.map({"high":2,"low":0,"medium":1})
y

0       2
1       2
2       0
3       0
4       1
       ..
1054    1
1055    2
1056    0
1057    2
1058    0
Name: Grade, Length: 1059, dtype: int64

In [71]:
y.value_counts()

Grade
0    429
1    374
2    256
Name: count, dtype: int64

In [72]:
from sklearn.tree import DecisionTreeClassifier
model1=DecisionTreeClassifier()

In [73]:
from sklearn.linear_model import LogisticRegression
model2=LogisticRegression(max_iter=1000,solver="liblinear")

In [74]:
from sklearn.model_selection import StratifiedKFold,cross_val_score
valid=StratifiedKFold(10)

In [75]:
result1=cross_val_score(model1,x,y,cv=valid)

In [76]:
result1

array([0.95283019, 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 0.99056604, 1.        ])

In [77]:
result1.mean()

0.9943396226415094

In [78]:
from sklearn.preprocessing import MinMaxScaler

In [79]:
scaler=MinMaxScaler((0,1))
x=scaler.fit_transform(x)
x

array([[0.55384615, 0.01785714, 1.        , ..., 1.        , 0.        ,
        0.93333333],
       [0.55384615, 0.03571429, 0.        , ..., 0.        , 1.        ,
        0.86666667],
       [0.84615385, 0.64285714, 1.        , ..., 1.        , 1.        ,
        0.4       ],
       ...,
       [0.        , 0.10714286, 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.58461538, 0.16071429, 1.        , ..., 1.        , 0.        ,
        0.66666667],
       [0.86153846, 0.375     , 0.        , ..., 1.        , 1.        ,
        1.        ]])

In [80]:
result2=cross_val_score(model2,x,y,cv=valid)
result2.mean()

0.7610961365678347

In [81]:
from sklearn.neighbors import KNeighborsClassifier
model3=KNeighborsClassifier(n_neighbors=7)


In [82]:
result3=cross_val_score(model3,x,y,cv=valid)


In [83]:
result3.mean()

0.9895867026055706

In [84]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test=tts(x,y,train_size=0.8,random_state=0)

In [85]:
x_train.shape,y_train.shape

((847, 7), (847,))

In [86]:
model1.fit(x_train,y_train)

DecisionTreeClassifier()

In [87]:
y_train_pred=model1.predict(x_train)
y_test_pred=model1.predict(x_test)

In [88]:
from sklearn.metrics import confusion_matrix,accuracy_score
print("Training")
print("Accuracy:",accuracy_score(y_train,y_train_pred)*100)
print(confusion_matrix(y_train,y_train_pred))

Training
Accuracy: 100.0
[[360   0   0]
 [  0 297   0]
 [  0   0 190]]


In [89]:
from sklearn.metrics import confusion_matrix,accuracy_score
print("Testing")
print("Accuracy:",accuracy_score(y_test,y_test_pred)*100)
print(confusion_matrix(y_test,y_test_pred))

Testing
Accuracy: 99.05660377358491
[[68  1  0]
 [ 0 77  0]
 [ 0  1 65]]


In [93]:
import pickle
pickle.dump(model1,open("/kaggle/working/dtmodel2.pkl","wb"))